In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [5]:
data=pd.read_excel("赛题1数据材料/赛题1数据集/购地-市场交易-土地转让.xlsx",index_col="企业编号").drop_duplicates(keep='first')
data

,成交时间,土地面积(公顷),土地用途,土地使用年限,土地级别,土地使用权类型,土地利用状况,转让方式,转让价格(万元)
企业编号,,,,,,,,,
1001,2010-07-14,0.018960,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.012060,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.049570,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2012-11-20,0.762853,商务金融用地,NaN,NaN,NaN,"1、该宗地原于2009年1月划拨给长沙市滨江新城建设开发有限责任公司作为基础设施用地,其中",NaN,NaN
1001,2012-11-26,0.762853,商务金融用地,NaN,NaN,NaN,"根据《长沙市城乡规划局规划设计条件书》(案卷编号20120172XDGH1),用途为商业金",NaN,NaN
1001,2014-09-04,0.764940,其他商服用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2015-03-26,8.065610,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2007-06-25 14:39:37,10.826100,工业用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2007-08-17 14:48:15,3.205400,工业用地,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days


dt = data[['成交时间']].applymap(to_day).groupby('企业编号').mean().T
dt = dt.reindex(columns=list(range(1001, 4001)))
dt

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
成交时间,-2705.603954,NaN,-2820.0,-1131.0,-501.0,NaN,-2238.75,-2137.555556,-2432.875,-2654.874832,...,-2399.0,NaN,-3258.666667,-2721.0,-2883.0,NaN,NaN,-2714.0,-2502.0,-1988.0


In [9]:
data["土地使用年限"].value_counts()

50                           293
40                           213
70                            90
39                            45
43                            36
36                            31
38                            30
35                            29
34                            27
37                            26
33                            24
30                            23
32                            18
28                            16
42                            15
41                            15
45                            14
46                            12
29                            11
49                            11
31                             9
47                             8
44                             7
48                             6
20                             6
69                             5
56                             5
24                             5
25                             4
23                             4
          

In [41]:
lst = data['土地使用年限'].value_counts()[data['土地使用年限'].value_counts() == 1].index


def to_land_use(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell


In [42]:
data=data.applymap(to_land_use)
data

,成交时间,土地面积(公顷),土地用途,土地使用年限,土地级别,土地使用权类型,土地利用状况,转让方式,转让价格(万元)
企业编号,,,,,,,,,
1001,2010-07-14,0.018960,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.012060,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.049570,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2012-11-20,0.762853,商务金融用地,NaN,NaN,NaN,"1、该宗地原于2009年1月划拨给长沙市滨江新城建设开发有限责任公司作为基础设施用地,其中",NaN,NaN
1001,2012-11-26,0.762853,商务金融用地,NaN,NaN,NaN,"根据《长沙市城乡规划局规划设计条件书》(案卷编号20120172XDGH1),用途为商业金",NaN,NaN
1001,2014-09-04,0.764940,其他商服用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2015-03-26,8.065610,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2007-06-25 14:39:37,10.826100,工业用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2007-08-17 14:48:15,3.205400,工业用地,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
data["土地使用年限"]

TypeError: cannot convert the series to <class 'float'>